In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [3]:
from google.colab import drive

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [7]:
df.shape

(18280, 48)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [9]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [11]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [12]:
df.features.head().values[0]

'[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [0]:
str_dict = df.features.head().values[0]

In [14]:
literal_eval(str_dict)[0]['key']

'Gender'

In [0]:
 def parse_features(x):
   output_dict = {}
   if str(x) == 'nan': return output_dict

   features = literal_eval(x.replace('\\"', '"'))
   for item in features:
     key = item['key'].lower().strip()
     value = item['value'][0].lower().strip()

     output_dict[key] = value

   return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [16]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [17]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [18]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_band color', 'feat_waist', 'feat_chain/necklace length (in.)',
       'feat_is energy star-certified', 'feat_case back',
       'feat_atpv arc rating', 'feat_watch band material',
       'feat_item spec attr - style (ebay)', 'feat_watch case shape',
       'feat_clothing type'],
      dtype='object', length=526)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [20]:
{k: v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

In [22]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [23]:
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [24]:
model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [25]:


model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat'], model)

(-57.172328503725836, 4.227341275008907)

In [0]:
# feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
result = run_model(feats, m)

In [46]:
# feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
print(result)
X = df[ feats ].values
y = df['prices_amountmin'].values

# m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)

m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

(-57.341289636097635, 4.21571342120217)


Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [31]:
 df['brand'].value_counts(normalize = True)

nike                    0.097210
puma                    0.033315
ralph lauren            0.028775
vans                    0.021116
new balance             0.020295
                          ...   
deadman wonderland      0.000055
therafirm               0.000055
exofficio               0.000055
adidas by raf simons    0.000055
under armor             0.000055
Name: brand, Length: 1732, dtype: float64

In [37]:
 df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'sport': 'soccer', 'country of manufacture': 'china', 'type': 'cleats', 'condition': 'new with box'},
       {'material': 'leather', 'gender': 'men', 'size': '12', 'color': 'white', 'model': '378336 120', 'manufacturer part number': '378336 120', 'brand': 'nike', 'age group': 'adult'},
       {'style': 'basketball shoes', 'pattern': 'solid', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'season': 'all-season', 'material': 'suede', 'gender': 'men', 'shoe size': '11', 'size': '11', 'color': 'gray', 'model': '378812 019', 'manufacturer part number': '378812 019', 'brand': 'nike', 'age group': 'adult'},
       {'sport': 'basketball', 'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/white/gym red', 'casual & dress shoe style': 'basketball', 'manufacturer part number': '684829001', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [0]:
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

In [35]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
m = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(feats, m)

(-57.16468774262015, 4.235810616060932)

In [39]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
toddler                4
mens                   4
boys'                  3
youth                  2
men||women             2
women ,�� unisex       2
women                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [41]:
feats_cat = [ x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_shoe category',
 'feat_catalog',
 'feat_clothing category',
 'feat_location - country',
 'feat_multi pack indicator',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_fit to_cat',
 'feat_frame shape_cat',
 'feat_sizebridge_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_used with equipment type_cat',
 'feat_fits model_cat',
 'feat_is dark sky-compliant_cat',
 'feat_chest size_cat',
 'feat_general warranty_cat',
 'feat_lining_cat',
 'feat_case tone_cat',
 'feat_profession_cat',
 'feat_case thickness_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_metal stamp_cat',
 'feat_genre_cat',
 'feat_lens_cat',
 'feat_is weather-resistant_cat',
 'feat_front_cat',
 'feat_item weight_cat',
 'feat_vendor description_cat',
 'feat_

In [0]:
#feats += feats_cat
#feats = list(set(feats))

feats = ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']

In [47]:
pwd


'/content/drive/My Drive/Colab Notebooks/dw_matrix'

In [52]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   day4.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	day5.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [49]:
cd matrix_one/

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_one


In [50]:
ls

dat5.ipynb  day3.ipynb  day4.ipynb


In [51]:
ls


day3.ipynb  day4.ipynb  day5.ipynb
